In [7]:
! pip install fastapi[all] uvicorn[standard] wait4it

In [1]:
from fastapi import FastAPI
import uvicorn

from pymongo import MongoClient
import pymongo

In [2]:

app = FastAPI()

def run():
    uvicorn.run(app)

In [3]:
# Code in this cell is just for (re)starting the API on a Process, and other compatibility stuff with Jupyter cells.
# Just ignore it!

from multiprocessing import Process
from wait4it import wait_for

_api_process = None

def start_api():
    """Stop the API if running; Start the API; Wait until API (port) is available (reachable)"""
    global _api_process
    if _api_process:
        _api_process.terminate()
        _api_process.join()
    
    _api_process = Process(target=run, daemon=True)
    _api_process.start()
    wait_for(port=8000)

def delete_route(method: str, path: str):
    """Delete the given route from the API. This must be called on cells that re-define a route"""
    [app.routes.remove(route) for route in app.routes if method in route.methods and route.path == path]

In [4]:
@app.get("/")
def get_root():
    return {"Hello": "World"}

@app.get('/instruments')
def get_instruments():
    global instruments_collection
    instruments = []
    for instrument in instruments_collection.find():
        del instrument['_id']
        instruments.append(instrument)
    return {'instruments': instruments}
    

In [5]:
client = MongoClient('mongodb://localhost:27017/')
db = client['kiteconnect']
stocks_actions_recommendations = db['stocks_actions_recommendations']
instruments_collection = db['instruments']

In [6]:
# ! sudo lsof -t -i tcp:8000 | xargs kill -9

[sudo] password for prathmesh: 


In [7]:
start_api()

INFO:     Started server process [3703]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
/home/prathmesh/miniconda3/envs/uaot-2/lib/python3.9/site-packages/pymongo/topology.py:162: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: https://pymongo.readthedocs.io/en/stable/faq.html#is-pymongo-fork-safe
  warnings.warn(


INFO:     127.0.0.1:57812 - "GET /instruments HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
pymongo.errors.AutoReconnect: connection pool paused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/prathmesh/miniconda3/envs/uaot-2/lib/python3.9/site-packages/uvicorn/protocols/http/httptools_impl.py", line 375, in run_asgi
    result = await app(self.scope, self.receive, self.send)
  File "/home/prathmesh/miniconda3/envs/uaot-2/lib/python3.9/site-packages/uvicorn/middleware/proxy_headers.py", line 75, in __call__
    return await self.app(scope, receive, send)
  File "/home/prathmesh/miniconda3/envs/uaot-2/lib/python3.9/site-packages/fastapi/applications.py", line 212, in __call__
    await super().__call__(scope, receive, send)
  File "/home/prathmesh/miniconda3/envs/uaot-2/lib/python3.9/site-packages/starlette/applications.py", line 112, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/home/prathmesh/miniconda3/envs/uaot-2/lib/python3.9/site-pa